In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D,ZeroPadding1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv1D,UpSampling1D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
#from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt

In [2]:
#!pip3 install mido

    100% |################################| 61kB 2.9MB/s ta 0:00:01


In [3]:
#!pip3 install music21

    100% |################################| 19.3MB 34kB/s eta 0:00:011   16% |#####                           | 3.2MB 7.8MB/s eta 0:00:03    19% |######                          | 3.8MB 10.2MB/s eta 0:00:02    24% |#######                         | 4.6MB 8.6MB/s eta 0:00:02    36% |###########                     | 7.1MB 6.2MB/s eta 0:00:02    38% |############                    | 7.4MB 8.5MB/s eta 0:00:02    51% |################                | 10.0MB 5.2MB/s eta 0:00:02    63% |####################            | 12.3MB 8.4MB/s eta 0:00:01    66% |#####################           | 12.9MB 15.5MB/s eta 0:00:01    95% |##############################  | 18.4MB 5.6MB/s eta 0:00:01    97% |############################### | 18.8MB 10.6MB/s eta 0:00:01
    100% |################################| 51kB 3.4MB/s eta 0:00:01
  Running setup.py bdist_wheel for music21 ... done
  Stored in directory: /root/.cache/pip/wheels/1a/0f/0e/f78ad56a2738f4ea5082dd4283d7b7cce18a8ba20c3e609b6c
Successfully b

In [3]:
from mido import MidiFile, MidiTrack, Message
from music21 import *
from IPython.display import Image
import pandas as pd
import numpy as np
import math
import glob

In [20]:
def recreate_midi(df_first_notes, speed=20000):
    # function to take a dataframe created by something like parse_notes() or a gan and return a midi
    
    # Can start by reverse scaling the note:
    df_reversed = df_first_notes.copy()
    df_reversed['note'] = round(df_reversed['note'] * 88 + 20)  # might want to have something more special than round()
    df_reversed.note = df_reversed.note.astype(int)
    df_reversed['velocity'] = 60  # create a uniform middling velocity

    # recreate the absolute time index and drop time_since_last (we'll recreate it with the stop signals)
    df_reversed['time_index'] = df_reversed.time_since_last.cumsum()
    df_reversed = df_reversed.drop(columns = 'time_since_last')

    # create a stop signal for each note at the appropriate time_index:
    for i in range(len(df_reversed)):
        stop_note = pd.DataFrame([[df_reversed.note[i], 0, 0, df_reversed.duration[i] + df_reversed.time_index[i]]],
                                 columns=['note', 'duration', 'velocity', 'time_index'])
        df_reversed = df_reversed.append(stop_note, ignore_index=True)
    df_reversed = df_reversed.sort_values('time_index').reset_index(drop=True)

    # recreate time_since last with the stop note signals
    df_reversed['time'] = [0] + [df_reversed.time_index[i+1] - df_reversed.time_index[i] 
                                 for i in range(len(df_reversed)-1)]
    # and now we don't need duration or time_index so can drop those
    df_reversed = df_reversed.drop(columns = {'time_index','duration'})

    # finally, we need to scale the time since last note appropriately:
    df_reversed['time'] = round(df_reversed['time'] * speed)
    df_reversed.time = df_reversed.time.astype(int)

    # finally, recreate the midi and return
    mid_remade = MidiFile()
    track = MidiTrack()
    mid_remade.tracks.append(track)
    track.append(Message('program_change', program=0, time=0))
    for i in range(len(df_reversed)):
        track.append(Message('note_on', note=df_reversed.note[i], velocity=df_reversed.velocity[i], time=df_reversed.time[i]))

    return mid_remade

In [5]:
!ls

Untitled.ipynb	song_long_generator.h5


In [4]:
# Loading the model
model = load_model('song_long_generator.h5')

In [11]:
# feeding random noise into the model
noise = tf.random.normal([1, 100])
generated_song = model(noise, training=False)

In [12]:
array = generated_song.numpy()

In [13]:
# Adjust for negative values in the model
array = np.where(array<0, 0.0001, array)

In [15]:
midi_df = pd.DataFrame(array[0],columns=["note", "duration",'time_since_last'])

In [16]:
midi_df.head()

,note,duration,time_since_last
0,0.456541,0.005347,0.009016
1,0.401147,0.000100,0.006355
2,0.479832,0.010043,0.000100
3,0.458423,0.005118,0.003468
4,0.307731,0.002722,0.002264


In [21]:
# Save the midi file
mid_remade = recreate_midi(midi_df)

In [22]:
mid_remade.save('mid_test_3.mid')